In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D,UpSampling1D
from keras.models import Model
from keras import backend as K

n_features = 4
sequence_length = 180
boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences_mix.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0"])
scaler = MinMaxScaler(feature_range=(0,1))
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_val = pd.read_csv("Data/Boat_nominal_data/Boat_sequence_mix_val.csv")
boat_val = boat_val.drop(columns=["Unnamed: 0"])
scaler = MinMaxScaler(feature_range=(0,1))
val_nom_data = scaler.fit_transform(boat_val)

print(normal_data.shape, val_nom_data.shape)
def prepare_sequences(data, batch_size):
    samples = []
    for i in range(0,data.shape[0], batch_size):
        sample = data[i:i+batch_size]	
        samples.append(sample)
    sequences = np.array(samples)
    trainX = np.reshape(sequences, (len(sequences), batch_size, n_features))
    return trainX


def prepare_data():    
    trainX_nominal = prepare_sequences(normal_data,sequence_length) 
    print(trainX_nominal.shape)
    
    valX_nominal = prepare_sequences(val_nom_data,sequence_length)
    print(valX_nominal.shape)

    return trainX_nominal, valX_nominal

trainX_nominal, valX_nominal = prepare_data()
n_sequences = len(trainX_nominal)


In [ ]:
# from keras.layers import LSTM, RepeatVector, TimeDistributed, Bidirectional
# from tensorflow_core.python.keras.models import Sequential
# 
# latent_dim = 20
# inputs = Input(shape=(sequence_length,n_features))
# x = inputs
# 
# for i in range(2):
#     x = LSTM(sequence_length, return_sequences=True)(x)
# 
# lstm_state_last, state_h, state_c = LSTM(sequence_length, return_state=True)(x)
# shape = K.int_shape(lstm_state_last)
# embeddings = Dense(latent_dim)(lstm_state_last)
# 
# encoder = Model(inputs, embeddings)
# encoder.summary()
# 
# latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
# x = Dense(sequence_length)(latent_inputs)
# x = RepeatVector(sequence_length)(x)
# 
# for i in range(1):
#     x = LSTM(sequence_length, return_sequences=True)(x)
# 
# outputs = LSTM(n_features, return_sequences=True)(x)
# 
# decoder = Model(latent_inputs, outputs)
# decoder.summary()
# 
# outputs = decoder(encoder.outputs)
# vae = Model(inputs, outputs)
# 
# vae.compile(optimizer='rmsprop', loss='mse')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 180, 4)       0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 90, 50)       450         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 45, 50)       5050        conv1d_5[0][0]                   
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 2250)         0           conv1d_6[0][0]                   
__________________________________________________________________________________________________
z_mean (De

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
latent_inputs (InputLayer)   (None, 20)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2250)              47250     
_________________________________________________________________
reshape_3 (Reshape)          (None, 45, 50)            0         
_________________________________________________________________
lambda_13 (Lambda)           (None, 45, 1, 50)         0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 90, 1, 50)         5050      
_________________________________________________________________
lambda_14 (Lambda)           (None, 90, 50)            0         
_________________________________________________________________
lambda_15 (Lambda)           (None, 90, 1, 50)         0         
__________

In [8]:
from keras.layers import Conv2DTranspose
from keras.losses import mse

intermediate_dimensions = 50
latent_dim = 20
filters = 50


def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=2, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        # by default, random_normal has mean=0 and std=1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon


inputs = Input(shape=(sequence_length, n_features))
x = inputs

for i in range(2):
    x = Conv1D(filters=filters,
               kernel_size=2,
               strides=2,
               padding='same')(x)

shape = K.int_shape(x)
x = Flatten()(x)
embeddings = Dense(intermediate_dimensions)(x)

z_mean = Dense(latent_dim, name='z_mean',)(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
x = Dense(shape[1]*shape[2])(latent_inputs)
x = Reshape((shape[1], shape[2]))(x)

for i in range(2):
    x = Conv1DTranspose(input_tensor=x,
                        filters=filters,
                        kernel_size=2,
                        last=False,
                        padding='same')


outputs = Conv1DTranspose(input_tensor=x,
                          filters=n_features,
                          kernel_size=2,
                          strides=1,
                          last=True,
                          padding='same')

decoder = Model(latent_inputs, outputs)
decoder.summary()

outputs = decoder(encoder.outputs[2])
reconstruction_loss = mse(K.flatten(inputs), K.flatten(outputs))
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
loss = K.mean(reconstruction_loss)


vae = Model(inputs, outputs, name='vae')
vae.add_loss(loss)

vae.compile(optimizer='rmsprop')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 180, 4)       0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 90, 50)       450         input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 45, 50)       5050        conv1d_5[0][0]                   
__________________________________________________________________________________________________
flatten_3 (Flatten)             (None, 2250)         0           conv1d_6[0][0]                   
__________________________________________________________________________________________________
z_mean (De

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
latent_inputs (InputLayer)   (None, 20)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2250)              47250     
_________________________________________________________________
reshape_3 (Reshape)          (None, 45, 50)            0         
_________________________________________________________________
lambda_13 (Lambda)           (None, 45, 1, 50)         0         
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 90, 1, 50)         5050      
_________________________________________________________________
lambda_14 (Lambda)           (None, 90, 50)            0         
_________________________________________________________________
lambda_15 (Lambda)           (None, 90, 1, 50)         0         
__________

In [9]:

vae.fit(x=trainX_nominal,
        epochs=300,
        batch_size=sequence_length)

Epoch 1/300


180/300 [=================>............] - ETA: 1s - loss: 0.3640

300/300 [==============================] - 2s 7ms/step - loss: 0.3545


Epoch 2/300


180/300 [=================>............] - ETA: 0s - loss: 0.3096

...



300/300 [==============================] - 0s 2ms/step - loss: 0.0018


Epoch 300/300


180/300 [=================>............] - ETA: 0s - loss: 0.0019

300/300 [==============================] - 0s 2ms/step - loss: 0.0018


In [10]:
def return_mask(num, labels):
    return np.squeeze(np.argwhere(labels == num))


labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")
labels = labels.drop(columns="Unnamed: 0")
labels = np.array(labels)
masks = [return_mask(num, labels)[:, 0] for num in range(0, 9)]
encodings = encoder.predict(trainX_nominal)

for i in range(3):
    for mask in masks:
        plt.scatter(x=encodings[i][:, 0][mask], 
                    y=encodings[i][:, 1][mask], alpha=0.5)
    plt.title('Encodings')
    plt.legend(labels=np.arange(0, 9))
    plt.show()


In [11]:

runs = []
for mask in masks:
    run_for_class = trainX_nominal[mask]
    print(run_for_class.shape)
    runs.append(run_for_class)

for j in range(9):    
    for i in runs[j]:
        run = np.reshape(i, (1,sequence_length, n_features))
        rec = vae.predict(run)
        rec = np.reshape(rec, (len(trainX_nominal[0]), n_features))
        reconstruction_df = pd.DataFrame(rec, columns=boat_csv.columns)
        plt.plot(reconstruction_df["Lon"], reconstruction_df["Lat"])
        plt.show()
        break


(189, 180, 4)
(13, 180, 4)
(16, 180, 4)
(11, 180, 4)
(15, 180, 4)
(14, 180, 4)
(12, 180, 4)
(15, 180, 4)
(15, 180, 4)


In [39]:
boat_strange = pd.read_csv("Data/Boat_nominal_data/Boat_unseen.csv")
boat_strange = boat_strange.drop(columns=["Unnamed: 0"])
scaler = MinMaxScaler()
boat_unseen_data = scaler.fit_transform(boat_strange)
boat_unseen_data = np.reshape(boat_unseen_data, (1,len(boat_unseen_data),n_features))


[-0.0489331] [0.93606246]


In [40]:
unseen_encoding = encoder.predict(boat_unseen_data)
print(unseen_encoding[2][:,0], unseen_encoding[2][:,1])


[-0.0489331] [0.93606246]
